# Demo: Basic ideas of the Attribute Bubbling agent

In [1]:
import sys
sys.executable

'/opt/anaconda3/envs/brainsim/bin/python'

## Import UKS

In [2]:
from uks import UKS

uks = UKS()
uks

successful UKS import


## Take a look at initial list of all things in UKS

<html>
    <br />
    <font size="4">
        We only care about things that have children, because leaf nodes cannot have any attributes bubbled into them.
    </font>
</html>

In [5]:
## Take a look at everything, just to be sure.
print([thing.Label for thing in uks.UKSList])

['Thing', 'Object', 'Action', 'unknownObject', 'is-a', 'inverseOf', 'hasProperty', 'is', 'RelationshipType', 'has-child', 'isExclusive', 'isTransitive', 'has', 'ClauseType', 'number', 'greaterThan', 'lessThan', 'digit', 'isSimilarTo', 'isCommutative', 'hasDigit', '.', '0', '2', '1', '3', '4', '5', '6', '7', '8', '9', 'some', 'many', 'none']


In [4]:
all_things_with_children = [thing for thing in uks.UKSList if thing.Children.Count != 0]

print([thing.Label for thing in all_things_with_children])

['Thing', 'Object', 'RelationshipType', 'has', 'number', 'digit']


## Introduce new things and attributes 

<html>
    <br />
    <font size="4">
        Add a bunch of statements, and see which relationships are also added.
    </font>
</html>

In [8]:
uks.AddStatement("Fido", "is-a", "dog")
uks.AddStatement("Spot", "is-a", "dog")
# uks.AddStatement("Trip is-a dog")

uks.AddStatement("Fido", "has", "leg", None, None, "4")
uks.AddStatement("Spot", "has", "leg", None, None, "4")

In [9]:
## Take a look at everything, just to be sure.
print([thing.Label for thing in uks.UKSList])

all_things_with_children = [thing for thing in uks.UKSList if thing.Children.Count != 0]

print("")
print([thing.Label for thing in all_things_with_children])

['Thing', 'Object', 'Action', 'unknownObject', 'is-a', 'inverseOf', 'hasProperty', 'is', 'RelationshipType', 'has-child', 'isExclusive', 'isTransitive', 'has', 'ClauseType', 'number', 'greaterThan', 'lessThan', 'digit', 'isSimilarTo', 'isCommutative', 'hasDigit', '.', '0', '2', '1', '3', '4', '5', '6', '7', '8', '9', 'some', 'many', 'none', 'Fido', 'dog', 'Spot', 'leg', 'leg.4']

['Thing', 'Object', 'unknownObject', 'RelationshipType', 'has', 'number', 'digit', 'dog', 'leg']


## Check all the relationships of added things

<html>
    <br />
    <font size="4">
        We will just use the labels, "dog", "Fido", and "Spot", for simplicity.
    </font>
</html>

In [10]:
print([rel.ToString() for rel in uks.Labeled("dog").Relationships])
print([rel.ToString() for rel in uks.Labeled("Fido").Relationships])
print([rel.ToString() for rel in uks.Labeled("Spot").Relationships])

['dog->has-child->Fido', 'dog->has-child->Spot']
['Fido->has->leg.4']
['Spot->has->leg.4']


## **Core Functionality**: Do attribute bubbling

In [11]:
thresh = 1

for parent in all_things_with_children:
    child_rels_to_be_removed = []
    reltype_count_dict = {}      # Dict[Tuple[str, str]: int]
    reltype_child_rel_dict = {}  # Dict[Tuple[str, str]: List[Tuple[Thing, Thing]]]
    for child in parent.Children:
        rels = child.Relationships
        for rel in rels:
            if rel.relType.Label not in ["has-child", "hasProperty", "inverseOf"]:
                reltype = (rel.relType.Label, rel.target.Label)
                if reltype not in reltype_count_dict.keys():
                    reltype_count_dict[reltype] = 1
                else:
                    reltype_count_dict[reltype] += 1
                if reltype not in reltype_child_rel_dict.keys():
                    reltype_child_rel_dict[reltype] = [(child, rel)]
                else:
                    reltype_child_rel_dict[reltype] += [(child, rel)]
    for reltype, count in reltype_count_dict.items():
        if count > thresh:
            uks.AddStatement(parent.Label, reltype[0], reltype[1])
            for child, rel in reltype_child_rel_dict[reltype]:
                child.RemoveRelationship(rel)

In [12]:
print([rel.ToString() for rel in uks.Labeled("dog").Relationships])
print([rel.ToString() for rel in uks.Labeled("Fido").Relationships])
print([rel.ToString() for rel in uks.Labeled("Spot").Relationships])

['dog->has-child->Fido', 'dog->has-child->Spot', 'dog->has->leg.4']
[]
[]
